<a href="https://colab.research.google.com/github/emely3h/Geospatial_ML/blob/feature%2Fphysics-jaccard-index/data_exploration/physics_jaccard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Physics Jaccard Index

Calculate the physics jaccard index which will be the main success metric.

### 0. Get Stats for each image

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! ls
%cd drive/MyDrive/MachineLearning/Geospatial_ML
! ls

drive  sample_data
/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning/Geospatial_ML
architecture.drawio  experiment_1_2.ipynb  prepare_data
data_exploration     experiments	   README.md
evaluation	     models		   requirements.txt


In [13]:
! git checkout feature/physics-jaccard-index
! git pull

Already on 'feature/physics-jaccard-index'
Your branch is up to date with 'origin/feature/physics-jaccard-index'.
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), 537 bytes | 9.00 KiB/s, done.
From https://github.com/emely3h/Geospatial_ML
   e6592a9..3ca7d9c  feature/physics-jaccard-index -> origin/feature/physics-jaccard-index
Updating e6592a9..3ca7d9c
Fast-forward
 models/helpers.py | 42 +++++++++++++++++++++---------------------
 1 file changed, 21 insertions(+), 21 deletions(-)


In [14]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from data_exploration.mask_stats import Mask_Stats
from prepare_data.create_mask import create_physical_mask
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from models.unet_model import unet_2d
from models.helpers import initialize_saved_data, jaccard_coef

### 1. Loading + Preparing Training Data

In [21]:
total_tiles = 11121
train_tiles = total_tiles // 100 * 60 +1
test_val_tiles = total_tiles // 100 * 20 +1
data_path = "../data_colab/256_256"

train_split_x = np.memmap(os.path.join(data_path, "train_split_x.npy"), mode="r", shape=(train_tiles, 256, 256, 5), dtype=np.float32)
train_split_y = np.memmap(os.path.join(data_path, "train_split_y.npy"), mode="r", shape=(train_tiles, 256, 256), dtype=np.float32)

train_stats = Mask_Stats(train_split_y)
train_stats.print_stats()


Shape: (6661, 256, 256)
Land pixels: 176919986  40.528 %
Valid pixels: 125877821  28.836 %
Invalid pixels: 133737489  30.636 %
Sum: 6661


### 2. Initializing training data

In [22]:
x_input,y_mask = initialize_saved_data(train_split_x, train_split_y, train_tiles)

Initializing saved data...
x_input shape: (6661, 256, 256, 5)
x_min: 0.0 x_max: 0.0

Copying saved data to x_input...
x_input shape: (6661, 256, 256, 5)
x_min: 0.0 x_max: 255.0

Initializing y_mask...
y_mask shape: (6661, 256, 256)
y_min: 0.0 y_max: 0.0

Copying saved data to y_mask...
y_mask shape: (6661, 256, 256)
y_min: 0.0 y_max: 2.0


### 3. Create Physical mask(training data)

In [23]:
pred_physical = create_physical_mask(x_input)

### 4. Encoding (training data)

In [24]:
y_one_hot = to_categorical(y_mask, num_classes=3)

### 5. Calculate Physics jaccard index(training data)

In [25]:
jaccard = jaccard_coef(y_one_hot, pred_physical)
print(jaccard)

tf.Tensor(0.7695766, shape=(), dtype=float32)


In [ ]:
# todo: over entire dataset, not only training and both for non-overlapping and overlapping => should be equal

In [ ]:


data_path = "../data_colab/256_200"
train_total_tiles = 11063
val_total_tiles = 3545
test_total_tiles = 3699

train_split_x = np.memmap(os.path.join(data_path, "train_split_x.npy"), mode="r", shape=(train_total_tiles, 256, 256, 5), dtype=np.uint8)
train_split_y = np.memmap(os.path.join(data_path, "train_split_y.npy"), mode="r", shape=(train_total_tiles, 256, 256), dtype=np.uint8)
val_split_x = np.memmap(os.path.join(data_path, "val_split_x.npy"), mode="r", shape=(val_total_tiles, 256, 256, 5), dtype=np.uint8)
val_split_y = np.memmap(os.path.join(data_path, "val_split_y.npy"), mode="r", shape=(val_total_tiles, 256, 256), dtype=np.uint8)
test_split_x = np.memmap(os.path.join(data_path, "test_split_x.npy"), mode="r", shape=(test_total_tiles, 256, 256, 5), dtype=np.uint8)
test_split_y = np.memmap(os.path.join(data_path, "test_split_y.npy"), mode="r", shape=(test_total_tiles, 256, 256), dtype=np.uint8)
